## VGG 16

- Checkout [this](https://neurohive.io/en/popular-networks/vgg16/) article for better Explanation.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## I don't know why but without running this cell the below code is shown an error. 
## Running all these imports again solved it.
## Will figure out soon.

In [2]:

real = "archive//real_and_fake_face//training_real"
fake = "archive//real_and_fake_face//training_fake"
datadir = "archive//real_and_fake_face"


real_path = os.listdir(real)
fake_path = os.listdir(fake)

In [3]:
training_data = []
IMG_SIZE = 224

## This means 0 will indicate Real facial Images and 1 to Fake facial Images.

categories = ["training_real" , "training_fake"]

def create_training_data():
    for category in categories:
        path = os.path.join(datadir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_UNCHANGED)
                new_array = cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                training_data.append([new_array,class_num])
            except:
                pass
create_training_data()

In [4]:
import random

np.random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

0
0
1
0
0
1
0
0
0
0


In [5]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y = np.array(y)
X = X/255.0 


In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
train_x = keras.utils.normalize(X_train,axis=1)
test_x = keras.utils.normalize(X_test, axis=1)

In [8]:
#vgg16_model = keras.applications.vgg16.VGG16()
vgg16_model = keras.models.load_model('upgraded_vgg16.h5')
vgg16_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

- **predictions (Dense)          (None, 1000)              4097000**

- Vgg-16 is trained for classification of 1000 different classes, but we do not need that.
- So we will remove that last layer and add one of our own.

In [9]:
type(vgg16_model)
## This is not a sequential model.

tensorflow.python.keras.engine.sequential.Sequential

In [11]:
vgg16_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hist = vgg16_model.fit(X_train,y_train, batch_size=20, epochs = 3, validation_split=0.1)

Epoch 1/3
74/74 [==============================] - 730s 10s/step - loss: nan - accuracy: 0.5313 - val_loss: nan - val_accuracy: 0.4756
Epoch 2/3
74/74 [==============================] - 718s 10s/step - loss: nan - accuracy: 0.5313 - val_loss: nan - val_accuracy: 0.4756
Epoch 3/3
74/74 [==============================] - 716s 10s/step - loss: nan - accuracy: 0.5313 - val_loss: nan - val_accuracy: 0.4756


In [12]:
vgg16_model.save('./vgg16.model', save_format='h5')

In [ ]:
epochs = 50
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
xc = range(epochs)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
val_loss, val_acc = vgg16_model.evaluate(X_test, y_test)
print(val_loss)
print(val_acc)

In [ ]:
predictions = vgg16_model.predict(X_test[[0]])
# predictions

In [ ]:
vgg16_model.predict_classes(x = X_test[[0]], verbose=0)
